In [5]:
from pyspark.sql import SparkSession
#from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# En pyspark nous auron donc 
# ma_variable = spark.table('')

# Exécutons maintenant ce code 
mon_dataframe = spark.table('formation.arrivees')

# Affichons notre dataframe afin de nous assurer de la récuperation de notre table
# Pour rappel nous avons deux manières d'afficher nos résultat :

# soit par la fonction display() qui va nous permettre de pouvoir mettre en forme nos résultats grâce aux options présentes en dessous 
display(mon_dataframe)

# soit par la fonction show() qui nous permet également d'afficher notre dataframe. On peut lui donner des paramètres lui indiquant la manière d'afficher le dataframe
mon_dataframe.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `formation`.`arrivees` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [formation, arrivees], [], false
